In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from scipy import signal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Dropout
from tensorflow.keras.layers import LSTM

import matplotlib.pyplot as plt
import scipy.io as sio

!pip install neurokit2
import neurokit2 as nk

import seaborn as sns

import pandas as pd
import numpy as np
import time
!pip install mne
import mne

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data1  = pd.read_csv("/content/drive/My Drive/normalized.csv")

In [ ]:
data2 = data1
data2

,Unnamed: 0,Electrode1_1,Electrode1_2,Electrode1_3,Electrode1_4,Electrode1_5,Electrode1_6,Electrode1_7,Electrode1_8,Electrode1_9,Electrode1_10,Electrode1_11,Electrode1_12,Electrode1_13,Electrode1_14,Electrode1_15,Electrode1_16,Electrode1_17,Electrode1_18,Electrode1_19,Electrode1_20,Electrode1_21,Electrode1_22,Electrode1_23,Electrode1_24,Electrode1_25,Electrode1_26,Electrode1_27,Electrode1_28,Electrode1_29,Electrode1_30,Electrode1_31,Electrode1_32,Electrode1_33,Electrode1_34,Electrode1_35,Electrode1_36,Electrode1_37,Electrode1_38,Electrode1_39,...,Electrode14_7653,Electrode14_7654,Electrode14_7655,Electrode14_7656,Electrode14_7657,Electrode14_7658,Electrode14_7659,Electrode14_7660,Electrode14_7661,Electrode14_7662,Electrode14_7663,Electrode14_7664,Electrode14_7665,Electrode14_7666,Electrode14_7667,Electrode14_7668,Electrode14_7669,Electrode14_7670,Electrode14_7671,Electrode14_7672,Electrode14_7673,Electrode14_7674,Electrode14_7675,Electrode14_7676,Electrode14_7677,Electrode14_7678,Electrode14_7679,Electrode14_7680,age,gender,participant,video,video_name,target_emotion,valence,arousal,dominance,class,valencehigh,arousalhigh
0,0,0.711454,0.816407,0.697264,0.623580,0.540883,0.697167,0.723525,0.664447,0.619359,0.600343,0.795455,0.692970,0.752745,0.748237,0.721244,0.812942,0.750268,0.713971,0.738795,0.846925,0.864148,0.750259,0.589057,0.649112,0.851468,0.727535,0.667166,0.513998,0.662799,0.776796,0.568155,0.500076,0.530692,0.573115,0.552580,0.448053,0.403018,0.367629,0.519017,...,0.444218,0.429233,0.417938,0.396732,0.375510,0.377816,0.389163,0.370040,0.328767,0.323363,0.343300,0.331148,0.334926,0.332230,0.325925,0.329387,0.333160,0.327799,0.319943,0.317603,0.326645,0.328819,0.315980,0.302303,0.298552,0.304833,0.296585,0.276508,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2,3,1,1
1,1,0.720999,0.824642,0.701456,0.629354,0.548922,0.707850,0.731817,0.670715,0.626261,0.608348,0.805158,0.699980,0.760697,0.756772,0.729119,0.823306,0.760343,0.721423,0.744670,0.850936,0.867150,0.756477,0.596873,0.656294,0.853543,0.726749,0.671016,0.519910,0.669217,0.780624,0.573094,0.505563,0.536576,0.578516,0.559275,0.457948,0.411667,0.375157,0.526581,...,0.485009,0.447577,0.438965,0.443438,0.438332,0.420980,0.392611,0.379576,0.363888,0.354684,0.351318,0.320224,0.334789,0.346021,0.342594,0.321363,0.313929,0.320664,0.326143,0.317346,0.304665,0.306543,0.308382,0.306202,0.300586,0.291296,0.290441,0.289939,22,male,1,2,D.O.A.,surprise,3,3,1,0,1,1
2,2,0.716593,0.817322,0.703663,0.635128,0.549264,0.698792,0.725657,0.675416,0.631039,0.604536,0.796221,0.700781,0.766376,0.756215,0.719793,0.815470,0.762701,0.724979,0.743327,0.846702,0.868651,0.761658,0.599035,0.653700,0.849689,0.731988,0.678717,0.520605,0.660331,0.777091,0.579415,0.509069,0.532065,0.572715,0.560654,0.462680,0.410195,0.368465,0.522799,...,0.425658,0.417648,0.405843,0.394956,0.385626,0.387274,0.378654,0.362965,0.343049,0.343962,0.363006,0.334972,0.310222,0.308233,0.291096,0.279795,0.281575,0.284084,0.274219,0.263598,0.266709,0.265663,0.255200,0.245400,0.243748,0.253295,0.250620,0.233153,22,male,1,3,The Hangover,amusement,5,4,4,0,1,1
3,3,0.720999,0.828911,0.705207,0.629726,0.547212,0.706456,0.733239,0.672870,0.626792,0.607395,0.805414,0.702183,0.761833,0.755288,0.728290,0.823812,0.758414,0.718713,0.743831,0.853832,0.871904,0.756891,0.594379,0.655097,0.858879,0.730417,0.671016,0.519388,0.668230,0.783274,0.576650,0.507087,0.536772,0.580116,0.561245,0.457088,0.410379,0.377457,0.529523,...,0.506221,0.471713,0.443245,0.439709,0.452884,0.451591,0.416420,0.383420,0.371029,0.385440,0.388557,0.342209,0.333697,0.345470,0.349370,0.325835,0.301585,0.305487,0.314131,0.308602,0.296613,0.289963,0.296550,0.302059,0.316022,0.311127,0.292331,0.288289,22,male,1,4,The Ring,fear,4,3,2,1,1,1
4,4,0.721733,0.825557,0.699912,0.627119,0.547725,0.706688,0.728500,0.667581,0.624491,0.606442,0.802605,0.698177,0.758425,0.753618,0.727254,0.822548,0.757985,0.718713,0.743999,0.852496,0.866900,0.755440,0.596707,0.655695,0.852060,0

In [ ]:
data2 = data2.drop(['Unnamed: 0'], axis = 1)
data2 = data2.drop(['age'], axis = 1)
data2 = data2.drop(['gender'],axis = 1)
data2 = data2.drop(['video'],axis =1)
data2 = data2.drop(['video_name'],axis =1)
data2 = data2.drop(['target_emotion'],axis =1)
data2 = data2.drop(['valence'],axis =1)
data2 = data2.drop(['arousal'],axis =1)
data2 = data2.drop(['dominance'],axis =1)
data2 = data2.drop(['class'],axis =1)
data2 = data2.drop(['arousalhigh'],axis =1)
data2 = data2.drop(['participant'],axis =1)
data2

,Electrode1_1,Electrode1_2,Electrode1_3,Electrode1_4,Electrode1_5,Electrode1_6,Electrode1_7,Electrode1_8,Electrode1_9,Electrode1_10,Electrode1_11,Electrode1_12,Electrode1_13,Electrode1_14,Electrode1_15,Electrode1_16,Electrode1_17,Electrode1_18,Electrode1_19,Electrode1_20,Electrode1_21,Electrode1_22,Electrode1_23,Electrode1_24,Electrode1_25,Electrode1_26,Electrode1_27,Electrode1_28,Electrode1_29,Electrode1_30,Electrode1_31,Electrode1_32,Electrode1_33,Electrode1_34,Electrode1_35,Electrode1_36,Electrode1_37,Electrode1_38,Electrode1_39,Electrode1_40,...,Electrode14_7642,Electrode14_7643,Electrode14_7644,Electrode14_7645,Electrode14_7646,Electrode14_7647,Electrode14_7648,Electrode14_7649,Electrode14_7650,Electrode14_7651,Electrode14_7652,Electrode14_7653,Electrode14_7654,Electrode14_7655,Electrode14_7656,Electrode14_7657,Electrode14_7658,Electrode14_7659,Electrode14_7660,Electrode14_7661,Electrode14_7662,Electrode14_7663,Electrode14_7664,Electrode14_7665,Electrode14_7666,Electrode14_7667,Electrode14_7668,Electrode14_7669,Electrode14_7670,Electrode14_7671,Electrode14_7672,Electrode14_7673,Electrode14_7674,Electrode14_7675,Electrode14_7676,Electrode14_7677,Electrode14_7678,Electrode14_7679,Electrode14_7680,valencehigh
0,0.711454,0.816407,0.697264,0.623580,0.540883,0.697167,0.723525,0.664447,0.619359,0.600343,0.795455,0.692970,0.752745,0.748237,0.721244,0.812942,0.750268,0.713971,0.738795,0.846925,0.864148,0.750259,0.589057,0.649112,0.851468,0.727535,0.667166,0.513998,0.662799,0.776796,0.568155,0.500076,0.530692,0.573115,0.552580,0.448053,0.403018,0.367629,0.519017,0.452125,...,0.635824,0.569541,0.638357,0.547847,0.554680,0.533226,0.561710,0.604823,0.600570,0.535027,0.482174,0.444218,0.429233,0.417938,0.396732,0.375510,0.377816,0.389163,0.370040,0.328767,0.323363,0.343300,0.331148,0.334926,0.332230,0.325925,0.329387,0.333160,0.327799,0.319943,0.317603,0.326645,0.328819,0.315980,0.302303,0.298552,0.304833,0.296585,0.276508,1
1,0.720999,0.824642,0.701456,0.629354,0.548922,0.707850,0.731817,0.670715,0.626261,0.608348,0.805158,0.699980,0.760697,0.756772,0.729119,0.823306,0.760343,0.721423,0.744670,0.850936,0.867150,0.756477,0.596873,0.656294,0.853543,0.726749,0.671016,0.519910,0.669217,0.780624,0.573094,0.505563,0.536576,0.578516,0.559275,0.457948,0.411667,0.375157,0.526581,0.463335,...,0.654707,0.621187,0.673206,0.554804,0.557924,0.565078,0.607447,0.614599,0.591586,0.538233,0.513864,0.485009,0.447577,0.438965,0.443438,0.438332,0.420980,0.392611,0.379576,0.363888,0.354684,0.351318,0.320224,0.334789,0.346021,0.342594,0.321363,0.313929,0.320664,0.326143,0.317346,0.304665,0.306543,0.308382,0.306202,0.300586,0.291296,0.290441,0.289939,1
2,0.716593,0.817322,0.703663,0.635128,0.549264,0.698792,0.725657,0.675416,0.631039,0.604536,0.796221,0.700781,0.766376,0.756215,0.719793,0.815470,0.762701,0.724979,0.743327,0.846702,0.868651,0.761658,0.599035,0.653700,0.849689,0.731988,0.678717,0.520605,0.660331,0.777091,0.579415,0.509069,0.532065,0.572715,0.560654,0.462680,0.410195,0.368465,0.522799,0.465904,...,0.616132,0.569301,0.632126,0.530927,0.527340,0.507104,0.542174,0.579405,0.573181,0.501603,0.443002,0.425658,0.417648,0.405843,0.394956,0.385626,0.387274,0.378654,0.362965,0.343049,0.343962,0.363006,0.334972,0.310222,0.308233,0.291096,0.279795,0.281575,0.284084,0.274219,0.263598,0.266709,0.265663,0.255200,0.245400,0.243748,0.253295,0.250620,0.233153,1
3,0.720999,0.828911,0.705207,0.629726,0.547212,0.706456,0.733239,0.672870,0.626792,0.607395,0.805414,0.702183,0.761833,0.755288,0.728290,0.823812,0.758414,0.718713,0.743831,0.853832,0.871904,0.756891,0.594379,0.655097,0.858879,0.730417,0.671016,0.519388,0.668230,0.783274,0.576650,0.507087,0.536772,0.580116,0.561245,0.457088,0.410379,0.377457,0.529523,0.462868,...,0.587537,0.571223,0.657281,0.536755,0.502549,0.502750,0.579867,0.624158,0.593996,0.523123,0.511884,0.506221,0.471713,0.443245,0.439709,0.452884,0.451591,0.416420,0.383420,0.371029,0.385440,0.388557,0.342209,0.333697,0.345470,0.349370,0.325835,0.3

In [ ]:
#Data prep for valence
#Valence is judged by alphas of F3 and F4
#F3 is electrode 3, F4 is elctrode 12

#F3 
target=data2['valencehigh']
target

0      1
1      1
2      1
3      1
4      1
      ..
409    0
410    0
411    1
412    0
413    0
Name: valencehigh, Length: 414, dtype: int64

In [ ]:
from sklearn import preprocessing

target
target = preprocessing.scale(target)
target

array([ 0.79772404,  0.79772404,  0.79772404,  0.79772404,  0.79772404,
       -1.25356634,  0.79772404, -1.25356634, -1.25356634,  0.79772404,
        0.79772404,  0.79772404,  0.79772404,  0.79772404, -1.25356634,
        0.79772404, -1.25356634,  0.79772404,  0.79772404, -1.25356634,
        0.79772404, -1.25356634, -1.25356634, -1.25356634,  0.79772404,
       -1.25356634, -1.25356634, -1.25356634,  0.79772404,  0.79772404,
        0.79772404, -1.25356634, -1.25356634, -1.25356634, -1.25356634,
        0.79772404,  0.79772404,  0.79772404,  0.79772404, -1.25356634,
        0.79772404,  0.79772404,  0.79772404, -1.25356634, -1.25356634,
        0.79772404,  0.79772404,  0.79772404,  0.79772404,  0.79772404,
        0.79772404,  0.79772404, -1.25356634,  0.79772404,  0.79772404,
        0.79772404,  0.79772404, -1.25356634,  0.79772404,  0.79772404,
        0.79772404, -1.25356634, -1.25356634, -1.25356634,  0.79772404,
        0.79772404,  0.79772404, -1.25356634, -1.25356634,  0.79

In [ ]:
coll =[]
for i in range(1,7681):
  for j in range(1,15):
    coll.append('Electrode' + str(j) +'_' + str(i) )
    
coll

['Electrode1_1',
 'Electrode2_1',
 'Electrode3_1',
 'Electrode4_1',
 'Electrode5_1',
 'Electrode6_1',
 'Electrode7_1',
 'Electrode8_1',
 'Electrode9_1',
 'Electrode10_1',
 'Electrode11_1',
 'Electrode12_1',
 'Electrode13_1',
 'Electrode14_1',
 'Electrode1_2',
 'Electrode2_2',
 'Electrode3_2',
 'Electrode4_2',
 'Electrode5_2',
 'Electrode6_2',
 'Electrode7_2',
 'Electrode8_2',
 'Electrode9_2',
 'Electrode10_2',
 'Electrode11_2',
 'Electrode12_2',
 'Electrode13_2',
 'Electrode14_2',
 'Electrode1_3',
 'Electrode2_3',
 'Electrode3_3',
 'Electrode4_3',
 'Electrode5_3',
 'Electrode6_3',
 'Electrode7_3',
 'Electrode8_3',
 'Electrode9_3',
 'Electrode10_3',
 'Electrode11_3',
 'Electrode12_3',
 'Electrode13_3',
 'Electrode14_3',
 'Electrode1_4',
 'Electrode2_4',
 'Electrode3_4',
 'Electrode4_4',
 'Electrode5_4',
 'Electrode6_4',
 'Electrode7_4',
 'Electrode8_4',
 'Electrode9_4',
 'Electrode10_4',
 'Electrode11_4',
 'Electrode12_4',
 'Electrode13_4',
 'Electrode14_4',
 'Electrode1_5',
 'Electrode

In [ ]:
data3 = data2[coll]

In [ ]:
data3

,Electrode1_1,Electrode2_1,Electrode3_1,Electrode4_1,Electrode5_1,Electrode6_1,Electrode7_1,Electrode8_1,Electrode9_1,Electrode10_1,Electrode11_1,Electrode12_1,Electrode13_1,Electrode14_1,Electrode1_2,Electrode2_2,Electrode3_2,Electrode4_2,Electrode5_2,Electrode6_2,Electrode7_2,Electrode8_2,Electrode9_2,Electrode10_2,Electrode11_2,Electrode12_2,Electrode13_2,Electrode14_2,Electrode1_3,Electrode2_3,Electrode3_3,Electrode4_3,Electrode5_3,Electrode6_3,Electrode7_3,Electrode8_3,Electrode9_3,Electrode10_3,Electrode11_3,Electrode12_3,...,Electrode3_7678,Electrode4_7678,Electrode5_7678,Electrode6_7678,Electrode7_7678,Electrode8_7678,Electrode9_7678,Electrode10_7678,Electrode11_7678,Electrode12_7678,Electrode13_7678,Electrode14_7678,Electrode1_7679,Electrode2_7679,Electrode3_7679,Electrode4_7679,Electrode5_7679,Electrode6_7679,Electrode7_7679,Electrode8_7679,Electrode9_7679,Electrode10_7679,Electrode11_7679,Electrode12_7679,Electrode13_7679,Electrode14_7679,Electrode1_7680,Electrode2_7680,Electrode3_7680,Electrode4_7680,Electrode5_7680,Electrode6_7680,Electrode7_7680,Electrode8_7680,Electrode9_7680,Electrode10_7680,Electrode11_7680,Electrode12_7680,Electrode13_7680,Electrode14_7680
0,0.711454,0.614954,0.604127,0.645316,0.673839,0.364362,0.352183,0.344177,0.462849,0.434443,0.512573,0.693429,0.486062,0.380296,0.816407,0.654289,0.744939,0.841341,0.657845,0.353627,0.344799,0.381989,0.399463,0.447540,0.559063,0.838309,0.461410,0.358417,0.697264,0.590988,0.565883,0.589517,0.658403,0.382620,0.352283,0.491120,0.428681,0.495297,0.493677,0.705733,...,0.365330,0.510204,0.422631,0.606268,0.369116,0.473710,0.685782,0.373729,0.098180,0.560297,0.150039,0.304833,0.496382,0.401923,0.449977,0.521354,0.511152,0.574780,0.281519,0.505459,0.544923,0.499054,0.204052,0.582903,0.271092,0.296585,0.501774,0.319195,0.445815,0.528942,0.532289,0.549952,0.249354,0.447339,0.665963,0.584743,0.121116,0.574468,0.195442,0.276508
1,0.720999,0.624939,0.592847,0.634541,0.662959,0.381649,0.358836,0.282414,0.388548,0.418718,0.495301,0.697381,0.451129,0.368496,0.824642,0.661197,0.748178,0.862863,0.668538,0.396373,0.375052,0.369629,0.379758,0.442073,0.560421,0.851057,0.436979,0.363214,0.701456,0.593165,0.570222,0.601655,0.673980,0.433204,0.388458,0.508880,0.416411,0.494231,0.499766,0.718858,...,0.299716,0.410382,0.321871,0.513222,0.330654,0.347718,0.556167,0.264742,0.081005,0.555197,0.128641,0.291296,0.502584,0.407241,0.425892,0.489615,0.482968,0.513196,0.263964,0.409389,0.468225,0.463738,0.195657,0.579071,0.261218,0.290441,0.496156,0.317517,0.437967,0.544112,0.544437,0.520854,0.270026,0.458097,0.662584,0.591052,0.124297,0.571838,0.199738,0.289939
2,0.716593,0.621529,0.582118,0.630051,0.657398,0.299202,0.337630,0.296087,0.395365,0.423924,0.487173,0.698617,0.462773,0.362667,0.817322,0.656880,0.728745,0.856356,0.663726,0.323834,0.354745,0.380812,0.377967,0.446278,0.547862,0.850721,0.448269,0.349723,0.703663,0.597519,0.546929,0.567724,0.650505,0.332036,0.346253,0.431694,0.375767,0.485213,0.476815,0.717476,...,0.249494,0.407276,0.241503,0.472086,0.311424,0.242560,0.362137,0.180968,0.065112,0.548448,0.097061,0.253295,0.497209,0.400900,0.393701,0.482614,0.435726,0.461388,0.235238,0.295852,0.289262,0.414127,0.181442,0.571641,0.231706,0.250620,0.501183,0.318236,0.358371,0.467066,0.415281,0.438409,0.207687,0.250849,0.514780,0.473473,0.096893,0.562089,0.150817,0.233153
3,0.720999,0.628105,0.635763,0.657588,0.686170,0.390957,0.392931,0.367280,0.501022,0.438781,0.532893,0.724061,0.501764,0.396361,0.828911,0.671848,0.804453,0.873373,0.684309,0.398964,0.397431,0.420247,0.449172,0.454689,0.593686,0.884938,0.483250,0.381952,0.705207,0.602090,0.589404,0.596414,0.681659,0.408560,0.397502,0.538251,0.506902,0.502860,0.514286,0.735206,...,0.341839,0.488021,0.370252,0.551420,0.358496,0.455853,0.630008,0.329402,0.100872,0.561422,0.140629,0.311127,0.507133,0.414400,0.439092,0.498016,0.469789,0.529814,0.265560,0.424127,0.485756,0.457852,0.200918,0.584389,0.258365,0.292331,0.511532,0.331656,0.456652,0.

In [ ]:
#Data prep to feed into LTSM
# 414 X 12 X (640*14) 
# 414 rows X 12 time frames X 4(theta, alpha, beta, gamma) 
data4 = np.array(data3)
def dataprep(df):
	x = []
	a = []
	q=[]
	row = 0
	while(row<414):
		column =0
		for timestep in range(0,12):
			z=[]
			for i in range(0,640):
				j=14
				y=[]
				while(j):
					a=df[row][column]
					y.append(a)
					column+=1
					j=j-1
				z.append(y)
			q.append(z)
		
		row+=1
		x.append(q)
		q=[]
	return x

dfq = dataprep(data4)
dfq = np.array(dfq)
#dfq = dfq.reshape(dfq[2].shape,dfq[0].shape, dfq[1].shape )
dfq.shape

(414, 12, 640, 14)

In [ ]:
dfa = dfq.reshape(dfq.shape[0],dfq.shape[1], 8960)
dfa.shape

(414, 12, 8960)

In [ ]:
#from keras.utils import to_categorical
#target = to_categorical(target)
target=data2['valencehigh']
target

0      1
1      1
2      1
3      1
4      1
      ..
409    0
410    0
411    1
412    0
413    0
Name: valencehigh, Length: 414, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

target = np.array(target, dtype = float)
x_Train, x_Test, y_Train, y_Test = train_test_split(dfa, target, test_size = 0.25, random_state = 5)
x_Train.shape

(310, 12, 8960)

In [ ]:
x_Test[0]

array([[0.71096427, 0.61714564, 0.62971114, ..., 0.55030684, 0.37762676,
        0.5580512 ],
       [0.59859576, 0.53657393, 0.59833472, ..., 0.70221328, 0.75323899,
        0.70155502],
       [0.42000835, 0.3596949 , 0.27267653, ..., 0.52728651, 0.25897003,
        0.31189189],
       ...,
       [0.65268456, 0.52344666, 0.58311651, ..., 0.71119029, 0.68928879,
        0.67368015],
       [0.59988442, 0.53785851, 0.77038363, ..., 0.51900716, 0.34219755,
        0.23975738],
       [0.33025005, 0.29985476, 0.57702764, ..., 0.57709865, 0.19687388,
        0.2737983 ]])

In [ ]:

#x_Train = x_Train.reshape(x_Train.shape[0],1,48)
#x_Test = x_Test.reshape(x_Test.shape[0],1,48)
#x_Train.shape

In [ ]:
model = Sequential()
model.add(LSTM(64, batch_input_shape=(None,12,8960),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, batch_input_shape=(None,12,8960),activation="sigmoid",return_sequences=False))
model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation= 'sigmoid'))
model.add(Dense(3, activation= 'softmax'))
#model.add(Dense(16))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_30 (LSTM)               (None, 12, 64)            2310400   
_________________________________________________________________
dropout_15 (Dropout)         (None, 12, 64)            0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 51        
Total params: 2,323,395
Trainable params: 2,323,395
Non-trainable params: 0
___________________________________________

In [ ]:
history = model.fit(x_Train, y_Train ,epochs=50, validation_data =(x_Test,y_Test))

Epoch 1/50
10/10 [==============================] - 2s 182ms/step - loss: 0.7755 - accuracy: 0.6000 - val_loss: 0.7376 - val_accuracy: 0.6250
Epoch 2/50
10/10 [==============================] - 1s 136ms/step - loss: 0.7433 - accuracy: 0.6065 - val_loss: 0.7200 - val_accuracy: 0.6250
Epoch 3/50
10/10 [==============================] - 1s 132ms/step - loss: 0.7295 - accuracy: 0.6065 - val_loss: 0.7083 - val_accuracy: 0.6250
Epoch 4/50
10/10 [==============================] - 1s 133ms/step - loss: 0.7125 - accuracy: 0.6065 - val_loss: 0.6996 - val_accuracy: 0.6250
Epoch 5/50
10/10 [==============================] - 1s 135ms/step - loss: 0.7055 - accuracy: 0.6065 - val_loss: 0.6945 - val_accuracy: 0.6250
Epoch 6/50
10/10 [==============================] - 1s 138ms/step - loss: 0.7009 - accuracy: 0.6065 - val_loss: 0.6928 - val_accuracy: 0.6250
Epoch 7/50
10/10 [==============================] - 1s 139ms/step - loss: 0.7037 - accuracy: 0.6065 - val_loss: 0.6844 - val_accuracy: 0.6250
Epoch 

In [ ]:
y_pred = model.predict_on_batch(x_Test)
print(y_pred)

[[0.40733543 0.00966256 0.583002  ]
 [0.40732464 0.00966281 0.5830125 ]
 [0.4073193  0.00966538 0.58301526]
 [0.407363   0.00966219 0.5829748 ]
 [0.40731925 0.00966485 0.58301586]
 [0.4073337  0.00966415 0.58300215]
 [0.40725544 0.00966262 0.58308196]
 [0.4073131  0.00966557 0.5830213 ]
 [0.4073381  0.00966455 0.58299726]
 [0.40732402 0.00966408 0.58301187]
 [0.40733376 0.00966398 0.5830023 ]
 [0.40733325 0.00966343 0.5830033 ]
 [0.4073027  0.00966562 0.58303165]
 [0.4073296  0.00966344 0.5830069 ]
 [0.40738338 0.00966228 0.58295435]
 [0.40732807 0.00966358 0.5830084 ]
 [0.4072723  0.0096677  0.58306   ]
 [0.4073444  0.00966284 0.58299273]
 [0.4072319  0.00966671 0.5831014 ]
 [0.4073846  0.00966164 0.58295375]
 [0.4073224  0.00966381 0.5830138 ]
 [0.4073236  0.00966343 0.58301294]
 [0.40732172 0.00966402 0.5830143 ]
 [0.4073394  0.00966425 0.58299637]
 [0.40731436 0.00966396 0.5830217 ]
 [0.40732598 0.00966456 0.5830095 ]
 [0.4073352  0.00966496 0.5829999 ]
 [0.40733412 0.00966443 0.58

In [ ]:
import seaborn as sns
import numpy as np
plt.plot(y_test.flatten(), marker='.', label='true')
plt.plot(y_pred.flatten(), 'r', label='predicted')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
pred = model.predict(x_Test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_Test,axis=1)
print(expected_classes.shape)
print(predict_classes.shape)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Training Accuracy: {correct}")

In [ ]:
model=Sequential()
model.add(LSTM(128,batch_input_shape = (414,12,4),activation = 'relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation ='relu'))
model.add(Dense(32,activation ='sigmoid'))
model.add(Dense(1,activation ='softmax'))


opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
model.compile(loss='sparse_categorical_crossentropy',optimizer =opt,metrics=['accuracy'])
#model.compile(loss='binary_crossentropy',optimizer =opt,metrics=['accuracy'])
history = model.fit(x_Train,y_Train,epochs=50, validation_data =(x_Test,y_Test))
y_pred = model.predict(x_Test)

In [ ]:
print(y_pred)